## Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom Ključevima, Semaforima i Uslovima

Svaka nit simulira rad jedne ćelije u sistemu i ima pristup stanjima svojih suseda. Ćelije nemaju pristup globalnom brojaču iteracija, već svaka ćelija interno vodi računa o broju trenutne iteracije. Pored matrice podataka potrebno je uvesti:
  - Listu brojača suseda koji su pročitali trenutnu vrednost (za svaku ćeliju po jedan brojač). Osmi (poslednji) sused koji pročita vrednost budi ćeliju kako bi mogla da upiše novu vrednost u matricu stanja (buđenje realizovati semaforom). Voditi računa o sinhronizaciji suseda koji menjaju vrednost brojača.
  - Uslov (Condition) na kome čekaju sve ćelije koje su upisale novu vrednost u matricu stanja, pre nego što pređu u sledeću iteraciju. 
  - Brojač ćelija, zaštićen ključem, koje su upisale novu vrednost u matricu stanja. Poslednja ćelija aktivira Uslov za sledeću iteraciju. Voditi računa o redosledu uzimanja i puštanja ključa za brojač i ključa za uslov.


In [1]:
# Imports:
import random, threading
import numpy as np

In [2]:
# Global variables:

# Safe to change values:
n_epoch = 50                    # Number of epochs ( iterations ) Conway's game of life have.
n = 30                          # Number of rows and columns ( matrix type: NxN ).

# Not safe to change values: #
n_neighbours = 8                            # Number of cell neighbours ( TOP_LEFT, TOP, TOP_RIGHT, LEFT, RIGHT, BOT_LEFT, BOT, BOT_RIGHT ).
epoch_condition = threading.Condition()     # 
cells_left = n**2                           # Condition on which every cell that have written it's state value in the steps list is waiting.
                                            # When epoch_condition reaches 0 we can go to the next epoch && epoch_condition should be reseted to the n * n value.
TOP_LEFT    =       0
TOP         =       1
TOP_RIGHT   =       2
LEFT        =       3
RIGHT       =       4
BOT_LEFT    =       5
BOT         =       6
BOT_RIGHT   =       7

# Global structs:
cells = None
steps = list()              # List of matrixes trough epoches - list[i] is the appereance of the values of the every cell state in the i-th epoch.


In [3]:
# Cell

class Cell( threading.Thread ):
    
    def __init__( self, x, y ):
        threading.Thread.__init__( self )

        self.state = random.randint( 0, 1 )                                     # Generating random state for first epoch.
        self.epoch = 0                                                          # Current epoch ( iteration ).
        self.neighbour_counters = np.zeros( ( n_neighbours, ), dtype = int )    # Neighbours who red the value of this concrete Cell ( ininitially all 0's ). 
        self.x = x                                                              # The x coordinate of cell in the matrix.
        self.y = y                                                              # The y coordinate of cell in the matrix.

        self.semaphore = threading.Semaphore( 0 )                               # TODO: descripton.

    
    def reset( self ):
        self.neighbour_counters *= 0

    def __str__( self ):
           return f'[ { self.x } ][ { self.y } ] - { self.state }'


    def get_alive_neighbours( self ):

        '''
        (1) Check neighbour.
        (2) Change value of neighbour_counter[ x ] list (x being the corresponding position)
            When self cell checks it's top right neighbour - for that neighbour we are oposite position - bottom left - so that is the position
            of the neighbour_counter we need to update.
        '''

        global cells, steps, cells_left
        
        alive = 0

        x = self.x
        y = self.y
        epoch = self.epoch

        # Bit of hardcoding.
        alive += steps[ epoch ][ ( x - 1 ) % n ][ ( y - 1 ) % n ]                           # (1) TOP_LEFT
        cells[ ( x - 1 ) % n ][ ( y - 1 ) % n ].neighbour_counters[ BOT_RIGHT ] = 1         # (2) YOUR BOT_RIGHT READ YOU
        cells[ ( x - 1 ) % n ][ ( y - 1 ) % n ].semaphore.release()
        
        alive += steps[ epoch ][ ( x - 1 ) % n ][ y ]                                       # (1) TOP
        cells[ ( x - 1 ) % n ][ y ].neighbour_counters[ BOT ] = 1                           # (2) YOUR BOT READ YOU
        cells[ ( x - 1 ) % n ][ y ].semaphore.release()

        alive += steps[ epoch ][ ( x - 1 ) % n ][ ( y + 1 ) % n ]                           # (1) TOP_RIGHT
        cells[ ( x - 1 ) % n ][ ( y + 1 ) % n ].neighbour_counters[ BOT_LEFT ] = 1          # (2) YOUR BOT_LEFT READ YOU
        cells[ ( x - 1 ) % n ][ ( y + 1 ) % n ].semaphore.release()

        alive += steps[ epoch ][ x ][ ( y - 1 ) % n ]                                       # (1) LEFT
        cells[ x ][ ( y - 1 ) % n ].neighbour_counters[ RIGHT ] = 1                         # (2) YOUR RIGHT READ YOU
        cells[ x ][ ( y - 1 ) % n ].semaphore.release()
        
        alive += steps[ epoch ][ x ][ ( y + 1 ) % n ]                                       # (1) RIGHT
        cells[ x ][ ( y + 1 ) % n ].neighbour_counters[ LEFT ] = 1                          # (2) YOUR LEFT READ YOU
        cells[ x ][ ( y + 1 ) % n ].semaphore.release()

        alive += steps[ epoch ][ ( x + 1 ) % n ][ ( y - 1 ) % n ]                           # (1) BOT_LEFT
        cells[ ( x + 1 ) % n ][ ( y - 1 ) % n ].neighbour_counters[ TOP_RIGHT ] = 1         # (2) YOUR TOP_RIGH READ YOU
        cells[ ( x + 1 ) % n ][ ( y - 1 ) % n ].semaphore.release()

        alive += steps[ epoch ][ ( x + 1 ) % n ][ y ]                                       # (1) BOT
        cells[ ( x + 1 ) % n ][ y ].neighbour_counters[ TOP ] = 1                           # (2) YOUR TOP READ YOU
        cells[ ( x + 1 ) % n ][ y ].semaphore.release()

        alive += steps[ epoch ][ ( x + 1 ) % n ][ ( y + 1 ) % n ]                           # (1) BOT_RIGHT
        cells[ ( x + 1 ) % n ][ ( y + 1 ) % n ].neighbour_counters[ TOP_LEFT ] = 1          # (2) YOUR TOP_LEFT READ
        cells[ ( x + 1 ) % n ][ ( y + 1 ) % n ].semaphore.release()

        return alive


    def run( self ):
        
        global cells, steps, cells_left

        # Every thread has to loop n_epoch times.
        for epoch in range( 0, n_epoch ):
            
            alive = self.get_alive_neighbours()

            # Halt until all neihbours read our state.
            for _ in range( n_neighbours ):
                self.semaphore.acquire()

            # Cell is alive.
            if self.state == 1:
                if alive < 2 or alive > 3:
                    self.state = 0
            
            # Cell is dead.
            else:
                if alive == 3:
                    self.state = 1
            
            self.reset()

            # Waiting for all other cells to finish.
            epoch_condition.acquire()

            cells_left -= 1

            if cells_left == 0:
                steps.append( np.array( [ [ cells[i][j].state for j in range( n ) ] for i in range( n ) ] ) )
                cells_left = n ** 2
                epoch_condition.notifyAll()    
            
            else:
                epoch_condition.wait()

            epoch_condition.release()
            
            self.epoch = epoch

In [4]:
# Main

def main():

    global cells, steps, cells_left

    # Generating cells with random state.
    cells = [ [ Cell( i, j ) for j in range( n ) ] for i in range( n ) ]

    steps = list()

    # Generating initial step matrix with cell states.
    steps.append( np.array( [ [ cells[i][j].state for j in range( n ) ] for i in range( n ) ] ) )
  
    for i in range( n ):
        for j in range( n ):
            cells[i][j].start()

    for i in range( n ):
        for j in range( n ):
            cells[i][j].join()
    

if __name__ == "__main__":
    main()

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

def animate( steps ):
  '''
  Animates the array of matrices - each matrix is a single state in simulation.
  '''
  
  def init():
    im.set_data( steps[0] )
    return [ im ]
  
  
  def animate( i ):
    im.set_data( steps[i] )
    return [ im ]


  im = plt.matshow( steps[0], interpolation = 'None', animated = True );
  
  anim = FuncAnimation( im.get_figure(), animate, init_func = init,
                  frames = len( steps ), interval = 1000, blit = True, repeat = False );
                  
  return anim

anim = animate( steps );
HTML( anim.to_html5_video() )